In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model # Import Model for the functional API approach
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D # Added GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import save_model, load_model # Ensure load_model is imported if you plan to load later
from tensorflow.keras.applications import MobileNetV2 # Import MobileNetV2
from tensorflow.keras.optimizers import Adam # Explicitly import Adam

import kagglehub # Import kagglehub

# Assuming 'path' variable from previous cell is available:
# path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
# print("Path to dataset files:", path)

# Paths - Construct paths using the downloaded dataset path
# The structure of the downloaded dataset is typically 'dataset_name/subset/category'
# We expect the data to be in a subdirectory named 'chest_xray' within the downloaded path
# Make sure the 'path' variable is defined, e.g., by running the kagglehub download cell before this one.
try:
    train_path = os.path.join(path, 'chest_xray', 'train')
    test_path = os.path.join(path, 'chest_xray', 'test')
    print(f"Using train path: {train_path}")
    print(f"Using test path: {test_path}")
except NameError:
    print("Error: The 'path' variable is not defined. Please run the cell that downloads the dataset using kagglehub.")
    # Exit the script if the path is not defined
    exit()


# Image settings
img_size = 150
batch_size = 8 # Reduced batch size for potentially lower memory usage

# Data Preprocessing with Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,      # Randomly rotate images by up to 20 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally by up to 10% of the width
    height_shift_range=0.1, # Randomly shift images vertically by up to 10% of the height
    shear_range=0.1,        # Apply shearing transformation
    zoom_range=0.1,         # Randomly zoom inside images
    horizontal_flip=True    # Randomly flip images horizontally
)
test_datagen = ImageDataGenerator(rescale=1./255) # No augmentation for test data

# Use the constructed paths
# Added error handling in case paths are not correctly set
try:
    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary'
    )

    test_gen = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary'
    )
except FileNotFoundError:
     print(f"Error: Directory not found. Please ensure the paths {train_path} and {test_path} exist.")
     # Exit the script if directories are not found
     exit()


# Load the pre-trained MobileNetV2 model
# include_top=False removes the classification layer
base_model = MobileNetV2(input_shape=(img_size, img_size, 3),
                         include_top=False,
                         weights='imagenet') # Use weights pre-trained on ImageNet

# Freeze the base model layers so they are not trained
# This keeps the pre-trained features
base_model.trainable = False

# Create a new model on top of the pre-trained model
model = Sequential([
    base_model, # Add the pre-trained base model
    GlobalAveragePooling2D(), # Use Global Average Pooling instead of Flatten
    Dense(128, activation='relu'),
    BatchNormalization(), # Optional: Add Batch Norm here too
    Dropout(0.5),
    Dense(1, activation='sigmoid') # Output layer for binary classification
])

# Compile the model
# Use a slightly lower learning rate for Adam when using transfer learning
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print model summary to see the layers
model.summary()


# Train the model (only the newly added layers will be trained initially)
print("Starting model training (transfer learning)...")
# Increased epochs again, might need fewer with transfer learning
history = model.fit(train_gen, validation_data=test_gen, epochs=10) # Started with 10 epochs, adjust as needed

# Optional: Fine-tuning
# After training the top layers, you can unfreeze some layers of the base model
# and train the whole model with a very low learning rate.
# This can sometimes improve accuracy further.

# Unfreeze the last few layers of the base model
# base_model.trainable = True
# # Let's fine-tune from this layer onwards
# fine_tune_at = 100 # Experiment with different layer indices

# # Freeze all layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False

# # Recompile the model with a very low learning rate
# model.compile(loss='binary_crossentropy',
#               optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001), # Using RMSprop with very low LR
#               metrics=['accuracy'])

# model.summary()

# # Continue training with fine-tuning
# print("Starting fine-tuning...")
# history_fine_tune = model.fit(train_gen,
#                               epochs=history.epoch[-1] + 5, # Train for an additional 5 epochs
#                               initial_epoch=history.epoch[-1],
#                               validation_data=test_gen)


# Save the final model
model.save('pneumonia_model_mobilenetv2.h5') # Save with a new name
print("Model saved as 'pneumonia_model_mobilenetv2.h5'")

# Check the model's accuracy on the test set
print("Evaluating model on test set...")
loss, accuracy = model.evaluate(test_gen)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Using train path: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/train
Using test path: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/test
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


<ipython-input-9-38ca1335110f>:71: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(img_size, img_size, 3),


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,593 (9.24 MB)

 Trainable params: 164,353 (642.00 KB)

 Non-trainable params: 2,258,240 (8.61 MB)

Starting model training (transfer learning)...
Epoch 1/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 188s 277ms/step - accuracy: 0.7001 - loss: 0.6285 - val_accuracy: 0.8285 - val_loss: 0.4003
Epoch 2/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 171s 263ms/step - accuracy: 0.8364 - loss: 0.3813 - val_accuracy: 0.8670 - val_loss: 0.3452
Epoch 3/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 214s 281ms/step - accuracy: 0.8842 - loss: 0.3029 - val_accuracy: 0.8590 - val_loss: 0.3309
Epoch 4/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 184s 282ms/step - accuracy: 0.8922 - loss: 0.2555 - val_accuracy: 0.8606 - val_loss: 0.3339
Epoch 5/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 177s 272ms/step - accuracy: 0.9008 - loss: 0.2471 - val_accuracy: 0.8798 - val_loss: 0.2826
Epoch 6/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 172s 264ms/step - accuracy: 0.9069 - loss: 0.2284 - val_accuracy: 0.8558 - val_loss: 0.2969
Epoch 7/10
652/652 ━━━━━━━━━━━━━━━━━━━━ 182s 279ms/step - accuracy: 0.9076 - loss: 0.2293 - val_accuracy: 0.8638 - val_loss: 0.2981
Epoch 8/10
652/652 ━━━━━━━━━━

Model saved as 'pneumonia_model_mobilenetv2.h5'
Evaluating model on test set...
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 192ms/step - accuracy: 0.8817 - loss: 0.2695
Test Loss: 0.2739
Test Accuracy: 0.8830


In [ ]:
from google.colab import files
files.download('pneumonia_model_mobilenetv2.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>